
# JAX ResNet CIFAR10

Connect to public google colab GPU runtime

In [1]:
!pip install flax

     |████████████████████████████████| 193 kB 5.1 MB/s 
     |████████████████████████████████| 118 kB 41.1 MB/s 
     |████████████████████████████████| 57 kB 4.3 MB/s 


In [2]:
#@title If connecting to TPU run this
# TPU setup : Boilerplate for connecting JAX to TPU.

import os
if 'google.colab' in str(get_ipython()) and 'COLAB_TPU_ADDR' in os.environ:
  # Make sure the Colab Runtime is set to Accelerator: TPU.
  import requests
  if 'TPU_DRIVER_MODE' not in globals():
    url = 'http://' + os.environ['COLAB_TPU_ADDR'].split(':')[0] + ':8475/requestversion/tpu_driver0.1-dev20191206'
    resp = requests.post(url)
    TPU_DRIVER_MODE = 1

  # The following is required to use TPU Driver as JAX's backend.
  from jax.config import config
  config.FLAGS.jax_xla_backend = "tpu_driver"
  config.FLAGS.jax_backend_target = "grpc://" + os.environ['COLAB_TPU_ADDR']
  print('Registered TPU:', config.FLAGS.jax_backend_target)
else:
  print('No TPU detected. Can be changed under "Runtime/Change runtime type".')

Registered TPU: grpc://10.48.142.146:8470


In [2]:
#@title imports

import io
import re

from functools import partial

import numpy as np

import matplotlib.pyplot as plt

import jax
import jax.numpy as jnp

import flax
import flax.linen as nn
import flax.optim as optim
import flax.jax_utils as flax_utils

# Assert that GPU is available
# assert 'Gpu' in str(jax.devices())

import tensorflow as tf
import tensorflow_datasets as tfds

import optax

from functools import partial
from typing import Any, Callable, Sequence, Tuple
ModuleDef = Any

import logging
from flax.training import train_state
logging.getLogger().setLevel(logging.INFO)

# Data from torch
from torchvision import transforms, datasets
from torch.utils.data import DataLoader

In [3]:
#@title ResNet model

class ResNetBlock(nn.Module):
    """ResNet block."""

    filters: int
    conv: ModuleDef
    norm: ModuleDef
    act: Callable
    strides: Tuple[int, int] = (1, 1)

    @nn.compact
    def __call__(
        self,
        x,
    ):
        residual = x
        y = self.conv(self.filters, (3, 3), self.strides, padding=((1, 1), (1, 1)))(x)
        y = self.norm()(y)
        y = self.act(y)
        y = self.conv(self.filters, (3, 3), padding=((1, 1), (1, 1)))(y)
        y = self.norm(scale_init=nn.initializers.zeros)(y)

        if residual.shape != y.shape:
            residual = self.conv(self.filters, (1, 1), self.strides, name="conv_proj")(
                residual
            )
            residual = self.norm(name="norm_proj")(residual)

        return self.act(residual + y)


class BottleneckResNetBlock(nn.Module):
    """Bottleneck ResNet block."""

    filters: int
    conv: ModuleDef
    norm: ModuleDef
    act: Callable
    strides: Tuple[int, int] = (1, 1)

    @nn.compact
    def __call__(self, x):
        residual = x
        y = self.conv(self.filters, (1, 1))(x)
        y = self.norm()(y)
        y = self.act(y)
        y = self.conv(self.filters, (3, 3), self.strides, padding=((1, 1), (1, 1)))(y)
        y = self.norm()(y)
        y = self.act(y)
        y = self.conv(self.filters * 4, (1, 1))(y)
        y = self.norm(scale_init=nn.initializers.zeros)(y)

        if residual.shape != y.shape:
            residual = self.conv(
                self.filters * 4, (1, 1), self.strides, name="conv_proj"
            )(residual)
            residual = self.norm(name="norm_proj")(residual)

        return self.act(residual + y)


class ResNet(nn.Module):
    """ResNetV1."""

    stage_sizes: Sequence[int]
    block_cls: ModuleDef
    num_classes: int
    num_filters: int = 64
    dtype: Any = jnp.float32
    act: Callable = nn.relu

    @nn.compact
    def __call__(self, x, train: bool = True):
        conv = partial(nn.Conv, use_bias=False, dtype=self.dtype)
        norm = partial(
            nn.BatchNorm,
            use_running_average=not train,
            momentum=0.9,
            epsilon=1e-5,
            dtype=self.dtype,
        )

        x = conv(
            self.num_filters, (3, 3), (1, 1), padding=((1, 1), (1, 1)), name="conv_init"
        )(x)
        x = norm(name="bn_init")(x)
        x = nn.relu(x)
        # x = nn.max_pool(x, (3, 3), strides=(2, 2), padding=((1, 1), (1, 1)))
        for i, block_size in enumerate(self.stage_sizes):
            for j in range(block_size):
                strides = (2, 2) if i > 0 and j == 0 else (1, 1)
                x = self.block_cls(
                    self.num_filters * 2 ** i,
                    strides=strides,
                    conv=conv,
                    norm=norm,
                    act=self.act,
                )(x)
        x = jnp.mean(x, axis=(1, 2))
        x = nn.Dense(self.num_classes, dtype=self.dtype)(x)
        x = jnp.asarray(x, self.dtype)
        return x


ResNet18 = partial(ResNet, stage_sizes=[2, 2, 2, 2], block_cls=ResNetBlock)
ResNet34 = partial(ResNet, stage_sizes=[3, 4, 6, 3], block_cls=ResNetBlock)
ResNet50 = partial(ResNet, stage_sizes=[3, 4, 6, 3], block_cls=BottleneckResNetBlock)
ResNet101 = partial(ResNet, stage_sizes=[3, 4, 23, 3], block_cls=BottleneckResNetBlock)
ResNet152 = partial(ResNet, stage_sizes=[3, 8, 36, 3], block_cls=BottleneckResNetBlock)
ResNet200 = partial(ResNet, stage_sizes=[3, 24, 36, 3], block_cls=BottleneckResNetBlock)


In [4]:
#@title Helpers - preprocess, train, eval steps

class ToArray:
    def __call__(self, x):
        x = np.asarray(x, dtype=np.float32)
        x /= 255.0
        return x

class ArrayNormalize:
    def __init__(self, mean, std):
        super().__init__()
        self.mean = mean
        self.std = std

    def __call__(self, x):
        mean = np.asarray(self.mean, dtype=x.dtype)
        std = np.asarray(self.std, dtype=x.dtype)
        if mean.ndim == 1:
            mean = mean.reshape(1, 1, -1)
        if std.ndim == 1:
            std = std.reshape(1, 1, -1)
        x -= mean
        x /= std
        return x

def array_collate(batch):
    imgs, targets = zip(*batch)
    return np.stack(imgs, axis=0), np.array(targets)

def shard(xs):
    return jax.tree_map(lambda x: x.reshape((jax.local_device_count(), -1) + x.shape[1:]) if len(x.shape) != 0 else x, xs)

class TrainState(train_state.TrainState):
    batch_stats: Any

def ce_loss(logits, labels):
    one_hot_labels = jax.nn.one_hot(labels, 10)
    xentropy = optax.softmax_cross_entropy(logits=logits, labels=one_hot_labels)
    return jnp.mean(xentropy)

def compute_metrics(logits, labels):
    loss = ce_loss(logits, labels)
    accuracy = jnp.mean(jnp.argmax(logits, -1) == labels)
    metrics = {
        'loss': loss,
        'accuracy': accuracy,
    }
    metrics = jax.lax.pmean(metrics, axis_name='batch')
    return metrics

def train_step(state, batch):
    imgs, targets = batch
    def loss_fn(params):
        logits, new_model_state = state.apply_fn(
            {'params': params, 'batch_stats': state.batch_stats},
            imgs,
            mutable=['batch_stats']
        )
        loss = ce_loss(logits, targets)
        weight_penalty_params = jax.tree_leaves(params)
        weight_decay = 5e-4
        weight_l2 = sum([jnp.sum(x ** 2) for x in weight_penalty_params if x.ndim > 1])
        weight_penalty = weight_decay * 0.5 * weight_l2
        loss = loss + weight_penalty
        return loss, (new_model_state, logits)

    grad_fn = jax.value_and_grad(loss_fn, has_aux=True)
    aux, grads = grad_fn(state.params)
    grads = jax.lax.pmean(grads, axis_name='batch')
    new_model_state, logits = aux[1]
    metrics = compute_metrics(logits, targets)

    new_state = state.apply_gradients(grads=grads, batch_stats=new_model_state['batch_stats'])
    return new_state, metrics

def eval_step(state, batch):
    imgs, targets = batch
    variables = {'params': state.params, 'batch_stats': state.batch_stats}
    logits = state.apply_fn(variables, imgs, train=False, mutable=False)
    return compute_metrics(logits, targets)

def stack_forest(forest):
    stack_args = lambda *args: np.stack(args)
    return jax.tree_multimap(stack_args, *forest)

def get_metrics(device_metrics):
    device_metrics = jax.tree_map(lambda x: x[0], device_metrics)
    metrics_np = jax.device_get(device_metrics)
    return stack_forest(metrics_np)

cross_replica_mean = jax.pmap(lambda x: jax.lax.pmean(x, 'x'), 'x')

def sync_batch_stats(state):
    return state.replace(batch_stats=cross_replica_mean(state.batch_stats))


In [5]:
BATCH_SIZE = 128
DATA_DIR = "data/"
EPOCHS = 30


In [6]:
def main():
    logging.info(f"JAX process: {jax.process_index()} / {jax.process_count()}")
    logging.info(f"JAX local devices: {jax.local_device_count()}")

    key = jax.random.PRNGKey(0)
    if BATCH_SIZE % jax.device_count() != 0:
        raise ValueError('Batch size must be divisible by the number of devices')
    LOCAL_BATCH_SIZE = BATCH_SIZE // jax.process_count()

    train_transform = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        ToArray(),
        ArrayNormalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])
    train_dset = datasets.CIFAR10(root=DATA_DIR, download=True, train=True, transform=train_transform)
    train_loader = DataLoader(train_dset, batch_size=LOCAL_BATCH_SIZE, shuffle=True, collate_fn=array_collate, num_workers=4, drop_last=True)

    val_transform = transforms.Compose([
        ToArray(),
        ArrayNormalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])
    val_dset = datasets.CIFAR10(root=DATA_DIR, download=True, train=False, transform=val_transform)
    val_loader = DataLoader(val_dset, batch_size=LOCAL_BATCH_SIZE, shuffle=False, collate_fn=array_collate, num_workers=4, drop_last=True)
    
    model = ResNet18(num_classes=10)

    @jax.jit
    def init(*args):
        return model.init(*args)
    vars = init({"params": key}, jnp.ones((1, 32, 32, 3)))
    params, batch_stats = vars["params"], vars["batch_stats"]
    optim = optax.rmsprop(learning_rate=0.0001, decay=1e-06, momentum=0.9)
    state = TrainState.create(apply_fn=model.apply, params=params, tx=optim, batch_stats=batch_stats)
    state = flax.jax_utils.replicate(state)

    p_train_step = jax.pmap(train_step, axis_name='batch')
    p_eval_step = jax.pmap(eval_step, axis_name='batch')
    for epoch in range(1, EPOCHS+1):
        logging.info(f"EPOCH: {epoch}")
        train_metrics, val_metrics = [], []
        for batch in train_loader:
            batch = shard(batch)
            imgs, tgts = batch
            state, metrics = p_train_step(state, batch)
            train_metrics.append(metrics) 
        train_metrics = get_metrics(train_metrics)
        train_summary = {
            f'train {k}': v
            for k, v in jax.tree_map(lambda x: x.mean(), train_metrics).items()
        }
        val_summary = {}
        if epoch % 5 == 0:   
            state = sync_batch_stats(state)    
            for batch in val_loader:
                batch = shard(batch)
                metrics = p_eval_step(state, batch)
                val_metrics.append(metrics)        
            val_metrics = get_metrics(val_metrics)
            val_summary = {
                f'val {k}': v
                for k, v in jax.tree_map(lambda x: x.mean(), val_metrics).items()
            }
        summary = {**train_summary, **val_summary}
        msg = "".join(["[{}] {:.5f} ".format(key, value) for key, value in summary.items()])
        logging.info(msg)

    jax.random.normal(key, ()).block_until_ready()
    logging.info("Completed, Cleaning up... Done!")

In [7]:
#@title Output from a GPU run
main()

INFO:absl:Unable to initialize backend 'tpu_driver': NOT_FOUND: Unable to find driver in registry given worker: 
INFO:absl:Unable to initialize backend 'tpu': INVALID_ARGUMENT: TpuPlatform is not available.
INFO:root:JAX process: 0 / 1
INFO:root:JAX local devices: 1


  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting data/cifar-10-python.tar.gz to data/


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Files already downloaded and verified


INFO:root:EPOCH: 1
INFO:root:[train accuracy] 0.41927 [train loss] 1.58606 
INFO:root:EPOCH: 2
INFO:root:[train accuracy] 0.58538 [train loss] 1.14783 
INFO:root:EPOCH: 3
INFO:root:[train accuracy] 0.65879 [train loss] 0.94817 
INFO:root:EPOCH: 4
INFO:root:[train accuracy] 0.71889 [train loss] 0.80188 
INFO:root:EPOCH: 5
INFO:root:[train accuracy] 0.74868 [train loss] 0.72249 [val accuracy] 0.72256 [val loss] 0.85418 
INFO:root:EPOCH: 6
INFO:root:[train accuracy] 0.77188 [train loss] 0.66075 
INFO:root:EPOCH: 7
INFO:root:[train accuracy] 0.78948 [train loss] 0.61210 
INFO:root:EPOCH: 8
INFO:root:[train accuracy] 0.80256 [train loss] 0.57567 
INFO:root:EPOCH: 9
INFO:root:[train accuracy] 0.81182 [train loss] 0.54749 
INFO:root:EPOCH: 10
INFO:root:[train accuracy] 0.82266 [train loss] 0.51819 [val accuracy] 0.74669 [val loss] 0.79857 
INFO:root:EPOCH: 11
INFO:root:[train accuracy] 0.82831 [train loss] 0.49704 
INFO:root:EPOCH: 12
INFO:root:[train accuracy] 0.83804 [train loss] 0.47616 
I

In [8]:
#@title Output from a TPU run
main()

INFO:absl:Unable to initialize backend 'gpu': FAILED_PRECONDITION: No visible GPU devices.
INFO:absl:Unable to initialize backend 'tpu': INVALID_ARGUMENT: TpuPlatform is not available.
INFO:root:JAX process: 0 / 1
INFO:root:JAX local devices: 8


  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting data/cifar-10-python.tar.gz to data/


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Files already downloaded and verified


AttributeError: ignored